In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import os
from shapely import wkt

# Get the city from command-line arguments
city = os.environ.get('CITY', 'Default City Name')
base_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/"

# Sanitize the city name
city_sanitized = city.replace(" ", "_").replace(",", "")
city_path = f"{base_path}{city_sanitized}"  # Use the sanitized city name here

# Load data
all_lts_df = pd.read_csv(f"{city_path}_all_lts.csv")

# Convert 'geometry' column from WKT strings to actual geometry objects
all_lts_df['geometry'] = all_lts_df['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
all_lts = gpd.GeoDataFrame(all_lts_df, geometry='geometry')

# Set the CRS for the GeoDataFrame
all_lts.crs = "EPSG:32632"

# Reproject to WGS84
all_lts_projected = all_lts.to_crs(epsg=4326)

gdf_nodes = pd.read_csv(f"{city_path}_gdf_nodes.csv", index_col=0)

# Create a colormap for slope classes
color_palette = ["green", "blue", "yellow", "red"]
lts_classes = [1, 2, 3, 4]
colors = dict(zip(lts_classes, color_palette))

# Calculate the mean of latitudes and longitudes
mean_latitude = all_lts_projected.geometry.apply(lambda geom: geom.centroid.y).mean()
mean_longitude = all_lts_projected.geometry.apply(lambda geom: geom.centroid.x).mean()

# Create a folium map centered on the mean of latitudes and longitudes
map_osm = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=11)

# # Add slope information to the map
for _, row in all_lts_projected.iterrows():
    color = colors.get((row['lts']), "#000000")  # default color is black
    folium.GeoJson(
        row['geometry'], 
        style_function=lambda _, color=color: {'color': color}  # use default argument to capture color
    ).add_to(map_osm)

# Create a custom legend HTML string for LTS levels
legend_html = '''
<div style="position: fixed; top: 10px; right: 10px; z-index: 1000; background-color: white; padding: 5px; border: 1px solid grey; font-size: 12px;">
<p><b>LTS Levels</b></p>
'''
for lts_class, color in colors.items():
    legend_html += f'<p><i class="fa fa-square" style="color:{color};"></i> LTS {lts_class}</p>'
legend_html += '</div>'

# Add the legend HTML to the map
map_osm.get_root().html.add_child(folium.Element(legend_html))

# Save the map
map_osm.save(f"{city_path}_lts_map.html")
# Display the map 
map_osm